In [1]:
pwd

'C:\\Users\\Baron\\Desktop\\EE_297_Repo\\EE_297\\ML_PATH_EE297\\EE297_env\\01_main\\03_lanes_code'

In [2]:
pwd

'C:\\Users\\Baron\\Desktop\\EE_297_Repo\\EE_297\\ML_PATH_EE297\\EE297_env\\01_main\\03_lanes_code'

In [3]:
pwd

'C:\\Users\\Baron\\Desktop\\EE_297_Repo\\EE_297\\ML_PATH_EE297\\EE297_env\\01_main\\03_lanes_code'

### IMPORTS

In [4]:
#1. Import Necessary Liobraries
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import cv2
import random
import shutil
from tqdm.auto import tqdm as notebook_tqdm
import tqdm
import tensorflow as tf

import pandas as pd
import numpy as np
from PIL import Image

import os

%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt


from keras.models import Sequential
from keras.layers import Input, AveragePooling2D, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.regularizers import l1

from keras.optimizers import Adam
from sklearn.model_selection import learning_curve
from sklearn.model_selection import StratifiedKFold
from sklearn.utils import shuffle

from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from tensorflow.keras.losses import SparseCategoricalCrossentropy

import seaborn as sns

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

#import torchvision.transforms as transforms

#import warnings
#warnings.filterwarnings("ignore")


### RESIZING IMAGE (SW + HW)

In [5]:
from PIL import Image

# Input and output paths
input_path = r"C:\Users\Baron\Desktop\EE_297_Repo\EE_297\hardware_imp\vitis_hls\lane_seg_hls\test_imgs\img_og.jpg"
output_path = r"C:\Users\Baron\Desktop\EE_297_Repo\EE_297\hardware_imp\vitis_hls\lane_seg_hls\test_imgs\img_og_224.jpg"

# Open the image
image = Image.open(input_path)

# Resize to 224x224
resized_image = image.resize((224, 224))

# Save the resized image
resized_image.save(output_path)

print(f"Image saved to {output_path}")

Image saved to C:\Users\Baron\Desktop\EE_297_Repo\EE_297\hardware_imp\vitis_hls\lane_seg_hls\test_imgs\img_og_224.jpg


### CONVERTS IMAGE TO .TXT FILE (HW) 

In [6]:
import cv2
import numpy as np
import os

# ==== USER INPUT ====
input_path = r"C:\Users\Baron\Desktop\EE_297_Repo\EE_297\hardware_imp\vitis_hls\lane_seg_hls\test_imgs\img_og_224.jpg"
output_basename = "img_rgb_224.txt"  # Output .txt filename
target_size = (224, 224)             # Resize target

# ==== LOAD IMAGE ====
img = cv2.imread(input_path)
if img is None:
    raise FileNotFoundError(f"Could not load image at: {input_path}")

img = cv2.resize(img, target_size)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert BGR → RGB
img = img.astype(np.float32) / 255.0        # Normalize to [0,1]

# ==== NORMALIZE WITH MEAN & STD (ImageNet-style) ====
mean = np.array([0.485, 0.456, 0.406])  # R, G, B
std  = np.array([0.229, 0.224, 0.225])

img = (img - mean[None, None, :]) / std[None, None, :]  # broadcast across H, W

# ==== WRITE TO TXT ====
output_dir = os.path.dirname(input_path)
output_path = os.path.join(output_dir, output_basename)

with open(output_path, "w") as f:
    for c in range(3):  # R, G, B
        for i in range(target_size[1]):
            for j in range(target_size[0]):
                f.write(f"{img[i, j, c]:.6f} ")
            f.write("\n")

print(f"[✓] Normalized image written to .txt for hardware input:\n{output_path}")

[✓] Normalized image written to .txt for hardware input:
C:\Users\Baron\Desktop\EE_297_Repo\EE_297\hardware_imp\vitis_hls\lane_seg_hls\test_imgs\img_rgb_224.txt


### MODEL CREATION (SW)
    1. LOAD IN TRAINED WEIGHTS FILE (SW)

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models

class LaneSegOnlyMobileNetV2(nn.Module):
    def __init__(self, pretrained=True, freeze_stem=False):
        super().__init__()
        # ---- Encoder: MobileNetV2 ----
        weights = models.MobileNet_V2_Weights.IMAGENET1K_V1 if pretrained else None
        m = models.mobilenet_v2(weights=weights)

        # Feature extractor (ends at last conv-bn-relu block)
        self.encoder = m.features              # output: (B, 1280, 7, 7) for 224x224
        enc_out_ch = 1280

        if freeze_stem:
            # freeze early, cheap speed/regularization trick
            for p in list(self.encoder.parameters())[:]:
                p.requires_grad = False

        # ---- Lightweight decoder (no skip connections) ----
        self.seg_head = nn.Sequential(
            nn.ConvTranspose2d(enc_out_ch, 256, kernel_size=2, stride=2),  # 7 -> 14
            nn.BatchNorm2d(256), nn.ReLU(inplace=True),

            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256), nn.ReLU(inplace=True),

            nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2),         # 14 -> 28
            nn.BatchNorm2d(128), nn.ReLU(inplace=True),

            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128), nn.ReLU(inplace=True),

            nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2),          # 28 -> 56
            nn.BatchNorm2d(64), nn.ReLU(inplace=True),

            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64), nn.ReLU(inplace=True),

            nn.ConvTranspose2d(64, 32, kernel_size=2, stride=2),           # 56 -> 112
            nn.BatchNorm2d(32), nn.ReLU(inplace=True),

            nn.Conv2d(32, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32), nn.ReLU(inplace=True),

            nn.Conv2d(32, 1, kernel_size=1),  # logits (B,1,112,112)
        )

    def forward(self, x):
        H, W = x.shape[-2:]
        feats = self.encoder(x)                 # (B,1280,7,7) at 224x224
        seg   = self.seg_head(feats)           # (B,1,112,112)
        seg   = F.interpolate(seg, size=(H, W), mode="bilinear", align_corners=False)
        return seg


In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = LaneSegOnlyMobileNetV2()
model.load_state_dict(torch.load("lane_seg_weights.pth"))
model.to(device)
model.eval()  # set to inference mode

C:\Users\Baron\AppData\Local\Temp\ipykernel_14072\2694822331.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("lane_seg_weights.pth"))


LaneSegOnlyMobileNetV2(
  (encoder): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2

In [9]:
for name, param in model.named_parameters():
    print(name, param.shape)

encoder.0.0.weight torch.Size([32, 3, 3, 3])
encoder.0.1.weight torch.Size([32])
encoder.0.1.bias torch.Size([32])
encoder.1.conv.0.0.weight torch.Size([32, 1, 3, 3])
encoder.1.conv.0.1.weight torch.Size([32])
encoder.1.conv.0.1.bias torch.Size([32])
encoder.1.conv.1.weight torch.Size([16, 32, 1, 1])
encoder.1.conv.2.weight torch.Size([16])
encoder.1.conv.2.bias torch.Size([16])
encoder.2.conv.0.0.weight torch.Size([96, 16, 1, 1])
encoder.2.conv.0.1.weight torch.Size([96])
encoder.2.conv.0.1.bias torch.Size([96])
encoder.2.conv.1.0.weight torch.Size([96, 1, 3, 3])
encoder.2.conv.1.1.weight torch.Size([96])
encoder.2.conv.1.1.bias torch.Size([96])
encoder.2.conv.2.weight torch.Size([24, 96, 1, 1])
encoder.2.conv.3.weight torch.Size([24])
encoder.2.conv.3.bias torch.Size([24])
encoder.3.conv.0.0.weight torch.Size([144, 24, 1, 1])
encoder.3.conv.0.1.weight torch.Size([144])
encoder.3.conv.0.1.bias torch.Size([144])
encoder.3.conv.1.0.weight torch.Size([144, 1, 3, 3])
encoder.3.conv.1.1.we

#### ==============================================================

### (encoder): Sequential( HW Conv -> FOLD (BatchNorm) -> Relu)
1. Save Weights File (HW)

In [10]:

#    It's a Conv2dNormActivation with (0)=Conv2d, (1)=BatchNorm2d, (2)=ReLU6
# encoder[0] = Conv2dNormActivation: [0]=Conv2d(bias=False), [1]=BatchNorm2d, [2]=ReLU6
enc0 = model.encoder[0]
conv: nn.Conv2d = enc0[0]
bn:   nn.BatchNorm2d = enc0[1]

assert isinstance(conv, nn.Conv2d)
assert isinstance(bn,   nn.BatchNorm2d)

print("Conv shape (O,I,K,K):", tuple(conv.weight.shape))
print("BN shapes:", tuple(bn.weight.shape), tuple(bn.bias.shape))

# Setting up BatchNorm Math Fold:
# Do folding in float32, cast once at the end
W32    = conv.weight.detach().cpu().numpy().astype(np.float32)
gamma32= bn.weight.detach().cpu().numpy().astype(np.float32)
beta32 = bn.bias.detach().cpu().numpy().astype(np.float32)
mean32 = bn.running_mean.detach().cpu().numpy().astype(np.float32)
var32  = bn.running_var.detach().cpu().numpy().astype(np.float32)
eps32  = np.float32(bn.eps)

s32    = gamma32 / np.sqrt(var32 + eps32)
W_fold = (W32 * s32[:, None, None, None]).astype(np.float16)
b_fold = (beta32 - mean32 * s32).astype(np.float16)

# If your HLS expects (K,K,IN_C,OUT_C) layout:
W_hls = np.transpose(W_fold, (2, 3, 1, 0))  # (K,K,I,O)

Conv shape (O,I,K,K): (32, 3, 3, 3)
BN shapes: (32,) (32,)


In [11]:
import os
import numpy as np

OUT_DIR = "weights"      # relative path in your project
CTYPE   = "data_t"       # ap_fixed typedef in your HLS

HEADER_INCLUDE = r'#include "C:\Users\Baron\Desktop\EE_297_Repo\EE_297\hardware_imp\vitis_hls\lane_seg_hls\lane_seg_top.h"'

def write_header_plain_numbers(array: np.ndarray, var_name: str, ctype: str, out_path: str):
    """
    Writes a header with:
      - #pragma once
      - your include path
      - declaration: data_t var[... ] = { ... };
      - numbers printed as plain literals (no 'f', no '(data_t)' casts)
    """
    array = np.asarray(array, dtype=np.float16)
    dims  = "".join(f"[{d}]" for d in array.shape)

    def fmt(a, indent=0):
        if a.ndim == 1:
            # no 'f' and no casts; compact scientific where needed
            return "{" + ",".join(f"{x:.8f}" for x in a.tolist()) + "}"
        body = ",\n".join((" "*(indent+2)) + fmt(x, indent+2) for x in a)
        return "{\n" + body + "\n" + (" "*indent) + "}"

    with open(out_path, "w") as f:
        f.write("#pragma once\n\n")
        f.write(f'{HEADER_INCLUDE}\n\n')
        f.write("// Auto-generated from encoder[0] with BN folded\n\n")
        # no 'const' here
        f.write(f"{ctype} {var_name}{dims} = ")
        f.write(fmt(array))
        f.write(";\n")

    # print full path (one-liner)
    print(os.path.abspath(out_path))

# ---- call these with your already-computed arrays ----
# W_hls: (K,K,IN_C,OUT_C), b_fold: (OUT_C,)
os.makedirs(OUT_DIR, exist_ok=True)
write_header_plain_numbers(W_hls, "conv0_w", CTYPE, os.path.join(OUT_DIR, "encoder_conv0_w.h"))
write_header_plain_numbers(b_fold, "conv0_b", CTYPE, os.path.join(OUT_DIR, "encoder_conv0_b.h"))


C:\Users\Baron\Desktop\EE_297_Repo\EE_297\ML_PATH_EE297\EE297_env\01_main\03_lanes_code\weights\encoder_conv0_w.h
C:\Users\Baron\Desktop\EE_297_Repo\EE_297\ML_PATH_EE297\EE297_env\01_main\03_lanes_code\weights\encoder_conv0_b.h


### (encoder): SW output of Conv Stage 0 Comparison
     1. Actual Model Software Output
     2. Custom Stage0 software output mimic data_t
     3. Print Debug Statements

In [12]:
### 
import torch
import torch.nn.functional as F
from torchvision import transforms
import numpy as np
from PIL import Image
import os

# -------------------------------
# CONFIGURATION
# -------------------------------
IMG_PATH = "C:/Users/Baron/Desktop/EE_297_Repo/EE_297/hardware_imp/vitis_hls/lane_seg_hls/test_imgs/img_rgb_224.txt"
WEIGHT_SAVE_DIR = "software_output"
os.makedirs(WEIGHT_SAVE_DIR, exist_ok=True)

# -------------------------------
# LOAD IMAGE (.txt format, channel-major: R→G→B blocks)
# -------------------------------
def load_txt_tensor(filepath, H=224, W=224, C=3):
    arr = np.loadtxt(filepath, dtype=np.float16)
    if arr.size != C * H * W:
        raise ValueError(f"Unexpected image size: got {arr.size} elements")

    arr = arr.reshape(C, H, W)          # (C,H,W), channel-major
    arr = np.expand_dims(arr, axis=0)   # (1,C,H,W) for torch
    return torch.tensor(arr, dtype=torch.float16)

# -------------------------------
# MAIN PIPELINE
# -------------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = LaneSegOnlyMobileNetV2(pretrained=False)
model.load_state_dict(torch.load("lane_seg_weights.pth", map_location=device))
model.to(device)
model.eval()

# Extract encoder[0] weights and BN params
enc0 = model.encoder[0]
conv: torch.nn.Conv2d = enc0[0]
bn:   torch.nn.BatchNorm2d = enc0[1]

# Do folding in float32, cast once at the end
W32    = conv.weight.detach().cpu().numpy().astype(np.float32)
gamma32= bn.weight.detach().cpu().numpy().astype(np.float32)
beta32 = bn.bias.detach().cpu().numpy().astype(np.float32)
mean32 = bn.running_mean.detach().cpu().numpy().astype(np.float32)
var32  = bn.running_var.detach().cpu().numpy().astype(np.float32)
eps32  = np.float32(bn.eps)

#======================== After this point we need float16 ============
s32     = gamma32 / np.sqrt(var32 + eps32)
W_fold  = (W32 * s32[:, None, None, None]).astype(np.float16)
b_fold  = (beta32 - mean32 * s32).astype(np.float16)

W_torch = torch.tensor(W_fold, dtype=torch.float16, device=device)
b_torch = torch.tensor(b_fold, dtype=torch.float16, device=device)

# Load image directly from .txt
x_q = load_txt_tensor(IMG_PATH).to(device)

print("x_q dtype:", x_q.dtype)
print("W_torch dtype:", W_torch.dtype)
print("b_torch dtype:", b_torch.dtype)

# -------------------------------
# Forward block (FP16)
# -------------------------------
with torch.no_grad():
    # Normal float16 conv
    y = F.conv2d(x_q, W_torch, bias=b_torch, stride=2, padding=1)
    print("conv2d output dtype:", y.dtype)
    
    # Apply ReLU6 and clamp
    y = torch.clamp(y, 0, 6)
    print("final output dtype:", y.dtype)
    

# Save results
y_np = y.squeeze(0).cpu().numpy()
np.save(os.path.join(WEIGHT_SAVE_DIR, "sw_encoder0_output.npy"), y_np)

with open(os.path.join(WEIGHT_SAVE_DIR, "sw_encoder0_output.txt"), "w") as f:
    for c in range(32):
        for i in range(112):
            for j in range(112):
                f.write(f"{y_np[c][i][j]:.6f} ")
            f.write("\n")
        f.write("\n")

print("✅ Software output with FP16 weights, input, and arithmetic saved.")


C:\Users\Baron\AppData\Local\Temp\ipykernel_14072\636227901.py:33: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("lane_seg_weights.pth", map

x_q dtype: torch.float16
W_torch dtype: torch.float16
b_torch dtype: torch.float16
conv2d output dtype: torch.float16
final output dtype: torch.float16
✅ Software output with FP16 weights, input, and arithmetic saved.


In [13]:
#3. Print Debug Statements
import torch.nn.functional as F

# -----------------------------------------
# Step 1: Create HLS-style padded buffer
# -----------------------------------------
def pad_like_hls_layout(x, pad=1):
    """
    Mimics HLS padded[y+PAD][x+PAD][c] = input[y][x][c]
    Input:  x (1,C,H,W)
    Output: (H+2*pad, W+2*pad, C) with channels innermost
    """
    _, C, H, W = x.shape
    out = np.zeros((H + 2*pad, W + 2*pad, C), dtype=np.float16)
    for c in range(C):
        out[pad:pad+H, pad:pad+W, c] = x[0, c, :, :].cpu().numpy()
    return out

PAD = 1
x_padded_hls = pad_like_hls_layout(x_q, pad=PAD)   # shape (226,226,3)

# -----------------------------------------
# Step 2: Debug the very first conv output: [oh=0, ow=0, oc=0]
# -----------------------------------------
STRIDE = 2
K = 3
iy = 0 * STRIDE
ix = 0 * STRIDE
oc = 0

# Bias (fp16)
bias_debug = np.float16(b_torch[oc].item())
print(f"DEBUG: bias[0] = {bias_debug:.6f}")

# First weight (fp16)
w_debug = np.float16(W_torch[oc, 0, 0, 0].item())
print(f"DEBUG: weights[0][0][0][0] = {w_debug:.6f}")

sum_debug = bias_debug

for ky in range(K):
    for kx in range(K):
        for ic in range(3):  # IN_C = 3
            # Input + weight (fp16)
            a = np.float16(x_padded_hls[iy + ky, ix + kx, ic])
            b = np.float16(W_torch[oc, ic, ky, kx].item())
            # Multiply and accumulate (fp16)
            p = np.float16(a * b)
            sum_debug = np.float16(sum_debug + p)
            print(f"DEBUG: a={a:.6f} * b={b:.6f} => {p:.6f} | sum={sum_debug:.6f}")

# Final ReLU6 (fp16)
sum_clamped = np.float16(max(0, min(6, sum_debug)))
print(f"DEBUG: output[0][0][0] = {sum_clamped:.6f}")


DEBUG: bias[0] = -0.022598
DEBUG: weights[0][0][0][0] = 0.006657
DEBUG: a=0.000000 * b=0.006657 => 0.000000 | sum=-0.022598
DEBUG: a=0.000000 * b=0.003578 => 0.000000 | sum=-0.022598
DEBUG: a=0.000000 * b=-0.016006 => -0.000000 | sum=-0.022598
DEBUG: a=0.000000 * b=-0.003366 => -0.000000 | sum=-0.022598
DEBUG: a=0.000000 * b=-0.004288 => -0.000000 | sum=-0.022598
DEBUG: a=0.000000 * b=-0.012825 => -0.000000 | sum=-0.022598
DEBUG: a=0.000000 * b=0.007576 => 0.000000 | sum=-0.022598
DEBUG: a=0.000000 * b=0.007786 => 0.000000 | sum=-0.022598
DEBUG: a=0.000000 * b=-0.006905 => -0.000000 | sum=-0.022598
DEBUG: a=0.000000 * b=0.018478 => 0.000000 | sum=-0.022598
DEBUG: a=0.000000 * b=0.010941 => 0.000000 | sum=-0.022598
DEBUG: a=0.000000 * b=-0.007534 => -0.000000 | sum=-0.022598
DEBUG: a=-1.500977 * b=-0.015869 => 0.023819 | sum=0.001221
DEBUG: a=-1.265625 * b=-0.020370 => 0.025787 | sum=0.027008
DEBUG: a=-1.002930 * b=-0.018280 => 0.018341 | sum=0.045349
DEBUG: a=-1.518555 * b=0.003067 => 

In [14]:

def load_txt_file(path):
    """
    Load numbers from a text file.
    Handles space, comma, or newline separated values.
    """
    with open(path, "r") as f:
        text = f.read()
    # Split by whitespace or commas
    numbers = text.replace(",", " ").split()
    return np.array([float(x) for x in numbers], dtype=np.float64)

def compare_outputs(hw_file, sw_file, dump_mismatches=False, tol=1e-5):
    # Load both files
    hw = load_txt_file(hw_file)
    sw = load_txt_file(sw_file)

    if hw.shape != sw.shape:
        raise ValueError(f"Shape mismatch: HW={hw.shape}, SW={sw.shape}")

    # Compute error metrics
    abs_err = np.abs(hw - sw)
    rel_err = abs_err / (np.abs(sw) + 1e-12)  # avoid div by zero

    mse = np.mean(abs_err**2)
    mae = np.mean(abs_err)
    max_err = np.max(abs_err)

    print("==== Quantization Error Report ====")
    print(f"Number of elements: {hw.size}")
    print(f"Mean Absolute Error (MAE): {mae:.6e}")
    print(f"Mean Squared Error (MSE): {mse:.6e}")
    print(f"Max Absolute Error: {max_err:.6e}")
    print(f"Mean Relative Error: {np.mean(rel_err):.6e}")
    print(f"Max Relative Error: {np.max(rel_err):.6e}")

    # Optionally dump a few mismatches
    if dump_mismatches:
        mismatches = np.where(abs_err > tol)[0]
        print(f"\nMismatches above tolerance {tol}: {len(mismatches)}")
        for idx in mismatches[:10]:  # show only first 10
            print(f"Idx {idx}: HW={hw[idx]}, SW={sw[idx]}, AbsErr={abs_err[idx]}")

    return mae, mse, max_err

if __name__ == "__main__":
    # Example usage
    hw_file = r"C:\Users\Baron\Desktop\EE_297_Repo\EE_297\hardware_imp\vitis_hls\lane_seg_hls\hw_output\encoder0_out.txt"
    sw_file = r"C:\Users\Baron\Desktop\EE_297_Repo\EE_297\ML_PATH_EE297\EE297_env\01_main\03_lanes_code\software_output\sw_encoder0_output.txt"
    compare_outputs(hw_file, sw_file, dump_mismatches=True, tol=1e-2)

==== Quantization Error Report ====
Number of elements: 401408
Mean Absolute Error (MAE): 5.512016e-04
Mean Squared Error (MSE): 1.482759e-06
Max Absolute Error: 1.953200e-02
Mean Relative Error: 5.433126e+04
Max Relative Error: 4.700000e+09

Mismatches above tolerance 0.01: 223
Idx 12613: HW=0.520996, SW=0.532227, AbsErr=0.011230999999999991
Idx 12742: HW=0.563965, SW=0.576172, AbsErr=0.012206999999999968
Idx 12951: HW=0.631836, SW=0.643066, AbsErr=0.011230000000000073
Idx 12970: HW=0.595703, SW=0.606445, AbsErr=0.01074200000000003
Idx 13064: HW=0.649902, SW=0.660156, AbsErr=0.010253999999999985
Idx 13248: HW=0.771973, SW=0.78418, AbsErr=0.012206999999999968
Idx 13364: HW=0.595703, SW=0.606934, AbsErr=0.011230999999999991
Idx 13370: HW=0.663086, SW=0.67334, AbsErr=0.010254000000000096
Idx 13392: HW=0.627441, SW=0.637695, AbsErr=0.010253999999999985
Idx 13399: HW=0.635742, SW=0.648438, AbsErr=0.01269599999999993


### (invertedresidual0): HW output of Conv Stage 1 
         1. Save Weights File

In [15]:
# 1) Get encoder stage (InvertedResidual at index 1)
ir0 = model.encoder[1]        # InvertedResidual
convseq = ir0.conv            # Sequential inside it

# ---- Depthwise ConvNormActivation ----
dw_conv: nn.Conv2d = convseq[0][0]   # Conv2d depthwise
dw_bn:   nn.BatchNorm2d = convseq[0][1]

assert isinstance(dw_conv, nn.Conv2d)
assert isinstance(dw_bn,   nn.BatchNorm2d)

print("Depthwise conv shape (O,I,K,K):", tuple(dw_conv.weight.shape))
print("Depthwise BN shapes:", tuple(dw_bn.weight.shape), tuple(dw_bn.bias.shape))

# Extract tensors
W_dw = dw_conv.weight.detach().cpu().numpy().astype("float32")   # (O,I,K,K)
gamma_dw = dw_bn.weight.detach().cpu().numpy().astype("float32")
beta_dw  = dw_bn.bias.detach().cpu().numpy().astype("float32")
mean_dw  = dw_bn.running_mean.detach().cpu().numpy().astype("float32")
var_dw   = dw_bn.running_var.detach().cpu().numpy().astype("float32")
eps_dw   = float(dw_bn.eps)

# Fold BN into depthwise conv
s_dw = (gamma_dw / np.sqrt(var_dw + eps_dw)).astype("float32")
W_dw_fold = (W_dw * s_dw[:, None, None, None]).astype("float32")
b_dw_fold = (beta_dw - mean_dw * s_dw).astype("float32")

# Layout for HLS: (K,K,IN_C,OUT_C)
W_dw_hls = np.transpose(W_dw_fold, (2, 3, 1, 0))

# ---- Pointwise Conv + BN ----
pw_conv: nn.Conv2d = convseq[1]
pw_bn:   nn.BatchNorm2d = convseq[2]

assert isinstance(pw_conv, nn.Conv2d)
assert isinstance(pw_bn,   nn.BatchNorm2d)

print("Pointwise conv shape (O,I,K,K):", tuple(pw_conv.weight.shape))
print("Pointwise BN shapes:", tuple(pw_bn.weight.shape), tuple(pw_bn.bias.shape))

# Extract tensors
W_pw = pw_conv.weight.detach().cpu().numpy().astype("float32")   # (O,I,K,K)
gamma_pw = pw_bn.weight.detach().cpu().numpy().astype("float32")
beta_pw  = pw_bn.bias.detach().cpu().numpy().astype("float32")
mean_pw  = pw_bn.running_mean.detach().cpu().numpy().astype("float32")
var_pw   = pw_bn.running_var.detach().cpu().numpy().astype("float32")
eps_pw   = float(pw_bn.eps)

# Fold BN into pointwise conv
s_pw = (gamma_pw / np.sqrt(var_pw + eps_pw)).astype("float32")
W_pw_fold = (W_pw * s_pw[:, None, None, None]).astype("float32")
b_pw_fold = (beta_pw - mean_pw * s_pw).astype("float32")

# Layout for HLS: (K,K,IN_C,OUT_C) — here K=1
W_pw_hls = np.transpose(W_pw_fold, (2, 3, 1, 0))


Depthwise conv shape (O,I,K,K): (32, 1, 3, 3)
Depthwise BN shapes: (32,) (32,)
Pointwise conv shape (O,I,K,K): (16, 32, 1, 1)
Pointwise BN shapes: (16,) (16,)


In [16]:
import os
import numpy as np

OUT_DIR = "weights"
CTYPE   = "data_t"  # ap_fixed typedef in HLS

HEADER_INCLUDE = r'#include "C:\Users\Baron\Desktop\EE_297_Repo\EE_297\hardware_imp\vitis_hls\lane_seg_hls\lane_seg_top.h"'

def write_header_plain_numbers(array: np.ndarray, var_name: str, ctype: str, out_path: str):
    array = np.asarray(array, dtype=np.float32)
    dims  = "".join(f"[{d}]" for d in array.shape)

    def fmt(a, indent=0):
        if a.ndim == 1:
            return "{" + ",".join(f"{x:.6f}" for x in a.tolist()) + "}"
        body = ",\n".join((" "*(indent+2)) + fmt(x, indent+2) for x in a)
        return "{\n" + body + "\n" + (" "*indent) + "}"

    with open(out_path, "w") as f:
        f.write("#pragma once\n\n")
        f.write(f"{HEADER_INCLUDE}\n\n")
        f.write("// Auto-generated with BN folded\n\n")
        f.write(f"{ctype} {var_name}{dims} = ")
        f.write(fmt(array))
        f.write(";\n")

    print("Wrote:", os.path.abspath(out_path))


os.makedirs(OUT_DIR, exist_ok=True)

# Depthwise
write_header_plain_numbers(W_dw_hls, "enc1_dw_w", CTYPE, os.path.join(OUT_DIR, "enc1_dw_w.h"))
write_header_plain_numbers(b_dw_fold, "enc1_dw_b", CTYPE, os.path.join(OUT_DIR, "enc1_dw_b.h"))

# Pointwise
write_header_plain_numbers(W_pw_hls, "enc1_pw_w", CTYPE, os.path.join(OUT_DIR, "enc1_pw_w.h"))
write_header_plain_numbers(b_pw_fold, "enc1_pw_b", CTYPE, os.path.join(OUT_DIR, "enc1_pw_b.h"))


Wrote: C:\Users\Baron\Desktop\EE_297_Repo\EE_297\ML_PATH_EE297\EE297_env\01_main\03_lanes_code\weights\enc1_dw_w.h
Wrote: C:\Users\Baron\Desktop\EE_297_Repo\EE_297\ML_PATH_EE297\EE297_env\01_main\03_lanes_code\weights\enc1_dw_b.h
Wrote: C:\Users\Baron\Desktop\EE_297_Repo\EE_297\ML_PATH_EE297\EE297_env\01_main\03_lanes_code\weights\enc1_pw_w.h
Wrote: C:\Users\Baron\Desktop\EE_297_Repo\EE_297\ML_PATH_EE297\EE297_env\01_main\03_lanes_code\weights\enc1_pw_b.h


### (invertedresidual0): SW output of Conv Stage 1

In [17]:
# -------------------------------
# CONFIGURATION
# -------------------------------
STAGE0_OUT_PATH = "software_output/sw_encoder0_output.npy"
WEIGHT_SAVE_DIR = "software_output"
os.makedirs(WEIGHT_SAVE_DIR, exist_ok=True)

# -------------------------------
# LOAD PREVIOUS STAGE OUTPUT
# -------------------------------
x = np.load(STAGE0_OUT_PATH).astype(np.float32)   # shape (32,112,112)
x = np.expand_dims(x, axis=0)                     # (1,32,112,112)
x_torch = torch.tensor(x, dtype=torch.float32)

# -------------------------------
# Load model + weights
# -------------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = LaneSegOnlyMobileNetV2(pretrained=False)
model.load_state_dict(torch.load("lane_seg_weights.pth", map_location=device))
model.to(device)
model.eval()

# Extract encoder[1] (InvertedResidual)
ir0 = model.encoder[1]
convseq = ir0.conv

# ---- Depthwise Conv (with BN folded) ----
dw_conv: torch.nn.Conv2d = convseq[0][0]
dw_bn:   torch.nn.BatchNorm2d = convseq[0][1]

W_dw = dw_conv.weight.detach().cpu().numpy().astype(np.float32)  # (32,1,3,3)
gamma_dw = dw_bn.weight.detach().cpu().numpy().astype(np.float32)
beta_dw  = dw_bn.bias.detach().cpu().numpy().astype(np.float32)
mean_dw  = dw_bn.running_mean.detach().cpu().numpy().astype(np.float32)
var_dw   = dw_bn.running_var.detach().cpu().numpy().astype(np.float32)
eps_dw   = float(dw_bn.eps)

#======================== After this point we need float16 ============
s_dw = gamma_dw / np.sqrt(var_dw + eps_dw)
W_dw_fold = W_dw * s_dw[:, None, None, None].astype(np.float16)
b_dw_fold = (beta_dw - mean_dw * s_dw).astype(np.float16)

# ---- Pointwise Conv (with BN folded) ----
pw_conv: torch.nn.Conv2d = convseq[1]
pw_bn:   torch.nn.BatchNorm2d = convseq[2]

W_pw = pw_conv.weight.detach().cpu().numpy().astype(np.float32)  # (16,32,1,1)
gamma_pw = pw_bn.weight.detach().cpu().numpy().astype(np.float32)
beta_pw  = pw_bn.bias.detach().cpu().numpy().astype(np.float32)
mean_pw  = pw_bn.running_mean.detach().cpu().numpy().astype(np.float32)
var_pw   = pw_bn.running_var.detach().cpu().numpy().astype(np.float32)
eps_pw   = float(pw_bn.eps)

#======================== After this point we need float16 ============
s_pw = gamma_pw / np.sqrt(var_pw + eps_pw)
W_pw_fold = W_pw * s_pw[:, None, None, None].astype(np.float16)
b_pw_fold = (beta_pw - mean_pw * s_pw).astype(np.float16)

# Convert to torch tensors
W_dw_torch = torch.tensor(W_dw_fold, dtype=torch.float16, device=device)
b_dw_torch = torch.tensor(b_dw_fold, dtype=torch.float16, device=device)
W_pw_torch = torch.tensor(W_pw_fold, dtype=torch.float16, device=device)
b_pw_torch = torch.tensor(b_pw_fold, dtype=torch.float16, device=device)
x_torch = x_torch.to(device).to(torch.float16)

# -------------------------------
# Run stage 1 (depthwise + pointwise)
# -------------------------------
with torch.no_grad():
    y = F.conv2d(x_torch, W_dw_torch, bias=b_dw_torch, stride=1, padding=1, groups=32)
    print("conv2d output dtype:", y.dtype)
    y = torch.clamp(y, 0, 6)  # ReLU6 after depthwise
    y = F.conv2d(y, W_pw_torch, bias=b_pw_torch, stride=1, padding=0)
    print("conv2d output dtype:", y.dtype)

# Save results
y_np = y.squeeze(0).cpu().numpy()  # (16,112,112)
np.save(os.path.join(WEIGHT_SAVE_DIR, "sw_encoder1_ir0_output.npy"), y_np)

with open(os.path.join(WEIGHT_SAVE_DIR, "sw_encoder1_ir0_output.txt"), "w") as f:
    for c in range(16):
        for i in range(112):
            for j in range(112):
                f.write(f"{y_np[c][i][j]:.6f} ")
            f.write("\n")
        f.write("\n")

print("✅ Software output for enc1_ir0 saved.")


C:\Users\Baron\AppData\Local\Temp\ipykernel_14072\854751219.py:20: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("lane_seg_weights.pth", map

conv2d output dtype: torch.float16
conv2d output dtype: torch.float16
✅ Software output for enc1_ir0 saved.


In [18]:
# === enc1_ir0 single-pixel debug (oh=0, ow=0), HLS-faithful ===
import numpy as np

# Pull raw arrays (keep dtype consistent with your forward pass: float16 or float32)
x_np  = x_torch.squeeze(0).detach().cpu().numpy()            # (32,112,112)
Wdw   = W_dw_torch.detach().cpu().numpy()                    # (32,1,3,3)  -> use [oc,0,ky,kx]
bdw   = b_dw_torch.detach().cpu().numpy()                    # (32,)
Wpw   = W_pw_torch.detach().cpu().numpy()                    # (16,32,1,1) -> use [oc,ic,0,0]
bpw   = b_pw_torch.detach().cpu().numpy()                    # (16,)

OUT_H = x_np.shape[1]    # 112
OUT_W = x_np.shape[2]    # 112
K = 3
oh = ow = 0

# ---- Depthwise: compute dw_out for ALL channels at (0,0) ----
dw_out00 = np.zeros((x_np.shape[0],), dtype=x_np.dtype)      # (32,)
for c in range(x_np.shape[0]):
    s = bdw[c]
    # DEBUG: channel 0 prints like HLS
    if c == 0:
        print(f"DEBUG DW: bias[{c}] = {float(bdw[c]):.6f}")
    for ky in range(K):
        for kx in range(K):
            iy = oh + ky - 1  # PAD=1 via bounds check
            ix = ow + kx - 1
            if 0 <= iy < OUT_H and 0 <= ix < OUT_W:
                a = x_np[c, iy, ix]
                b = Wdw[c, 0, ky, kx]    # depthwise weight for this channel
                p = a * b
                s = s + p
                if c == 0:
                    print(f"DW DEBUG: a={float(a):.6f} * b={float(b):.6f} => {float(p):.6f} | sum={float(s):.6f}")
    # ReLU6
    if s < 0:   s = type(s)(0)
    elif s > 6: s = type(s)(6)
    dw_out00[c] = s
    if c == 0:
        print(f"DW DEBUG: output[{oh}][{ow}][{c}] after ReLU6 = {float(s):.6f}")

# ---- Pointwise: oc=0 at same pixel, accumulate across ALL ic ----
oc_pw = 0
sum_pw = bpw[oc_pw]
print(f"PW DEBUG: bias[{oc_pw}] = {float(sum_pw):.6f}")
for ic in range(x_np.shape[0]):          # 32 channels
    a = dw_out00[ic]
    b = Wpw[oc_pw, ic, 0, 0]
    p = a * b
    sum_pw = sum_pw + p
    if ic < 3:  # match your HLS printf subset
        print(f"PW DEBUG: ic={ic}, a={float(a):.6f}, b={float(b):.6f}, p={float(p):.6f}, sum={float(sum_pw):.6f}")

print(f"PW DEBUG: output[{oh}][{ow}][{oc_pw}] final = {float(sum_pw):.6f}")


DEBUG DW: bias[0] = -0.014267
DW DEBUG: a=0.189087 * b=0.028519 => 0.005394 | sum=-0.008873
DW DEBUG: a=0.154785 * b=0.760254 => 0.117676 | sum=0.108826
DW DEBUG: a=0.227295 * b=-0.062988 => -0.014320 | sum=0.094482
DW DEBUG: a=0.180054 * b=0.057465 => 0.010345 | sum=0.104858
DW DEBUG: output[0][0][0] after ReLU6 = 0.104858
PW DEBUG: bias[0] = -2.091797
PW DEBUG: ic=0, a=0.104858, b=-0.010025, p=-0.001051, sum=-2.093750
PW DEBUG: ic=1, a=1.621094, b=0.081299, p=0.131836, sum=-1.961914
PW DEBUG: ic=2, a=2.710938, b=-0.066345, p=-0.179810, sum=-2.142578
PW DEBUG: output[0][0][0] final = -1.787109


sw = np.loadtxt(r"C:\Users\Baron\Desktop\EE_297_Repo\EE_297\ML_PATH_EE297\EE297_env\01_main\03_lanes_code\software_output\sw_encoder1_ir0_output.txt")
hw = np.loadtxt(r"C:\Users\Baron\Desktop\EE_297_Repo\EE_297\hardware_imp\vitis_hls\lane_seg_hls\hw_output\enc0_ir0_out.txt")

print(sw.shape, hw.shape)
sw = sw.reshape(16, 112, 112)
hw = hw.reshape(16, 112, 112)

print(np.allclose(sw[0], hw[0], atol=1e-3))  # compare just channel 0
print(np.max(np.abs(sw - hw)))               # worst-case diff

### (invertedresidual1): HW output of Conv Stage 2
     1. Save Weights File

In [19]:
# 1) Get encoder stage (InvertedResidual at index 2 for enc2_ir1)
ir1 = model.encoder[2]        # InvertedResidual (your "enc2_ir1")
convseq = ir1.conv            # Sequential inside it

# ---- Expansion Conv + BN (1x1, 16→96) ----
exp_conv: nn.Conv2d = convseq[0][0]
exp_bn:   nn.BatchNorm2d = convseq[0][1]

W_exp = exp_conv.weight.detach().cpu().numpy().astype("float32")  # (96,16,1,1)
gamma_exp = exp_bn.weight.detach().cpu().numpy().astype("float32")
beta_exp  = exp_bn.bias.detach().cpu().numpy().astype("float32")
mean_exp  = exp_bn.running_mean.detach().cpu().numpy().astype("float32")
var_exp   = exp_bn.running_var.detach().cpu().numpy().astype("float32")
eps_exp   = float(exp_bn.eps)

s_exp = (gamma_exp / np.sqrt(var_exp + eps_exp)).astype("float32")
W_exp_fold = (W_exp * s_exp[:, None, None, None]).astype("float32")
b_exp_fold = (beta_exp - mean_exp * s_exp).astype("float32")

# Layout for HLS: (K,K,IN_C,OUT_C) → (1,1,16,96)
W_exp_hls = np.transpose(W_exp_fold, (2, 3, 1, 0))

# ---- Depthwise Conv + BN (3x3, stride=2, groups=96) ----
dw_conv: nn.Conv2d = convseq[1][0]
dw_bn:   nn.BatchNorm2d = convseq[1][1]

W_dw = dw_conv.weight.detach().cpu().numpy().astype("float32")  # (96,1,3,3)
gamma_dw = dw_bn.weight.detach().cpu().numpy().astype("float32")
beta_dw  = dw_bn.bias.detach().cpu().numpy().astype("float32")
mean_dw  = dw_bn.running_mean.detach().cpu().numpy().astype("float32")
var_dw   = dw_bn.running_var.detach().cpu().numpy().astype("float32")
eps_dw   = float(dw_bn.eps)

s_dw = (gamma_dw / np.sqrt(var_dw + eps_dw)).astype("float32")
W_dw_fold = (W_dw * s_dw[:, None, None, None]).astype("float32")
b_dw_fold = (beta_dw - mean_dw * s_dw).astype("float32")

# Layout for HLS: (K,K,IN_C,OUT_C) — depthwise has groups=96, so IN=1 per channel
W_dw_hls = np.transpose(W_dw_fold, (2, 3, 1, 0))  # (3,3,1,96)

# ---- Projection Conv + BN (1x1, 96→24) ----
pw_conv: nn.Conv2d = convseq[2]
pw_bn:   nn.BatchNorm2d = convseq[3]

W_pw = pw_conv.weight.detach().cpu().numpy().astype("float32")  # (24,96,1,1)
gamma_pw = pw_bn.weight.detach().cpu().numpy().astype("float32")
beta_pw  = pw_bn.bias.detach().cpu().numpy().astype("float32")
mean_pw  = pw_bn.running_mean.detach().cpu().numpy().astype("float32")
var_pw   = pw_bn.running_var.detach().cpu().numpy().astype("float32")
eps_pw   = float(pw_bn.eps)

s_pw = (gamma_pw / np.sqrt(var_pw + eps_pw)).astype("float32")
W_pw_fold = (W_pw * s_pw[:, None, None, None]).astype("float32")
b_pw_fold = (beta_pw - mean_pw * s_pw).astype("float32")

# Layout for HLS: (K,K,IN_C,OUT_C) → (1,1,96,24)
W_pw_hls = np.transpose(W_pw_fold, (2, 3, 1, 0))

In [20]:
import os
import numpy as np

OUT_DIR = "weights"
CTYPE   = "data_t"  # ap_fixed typedef in HLS

HEADER_INCLUDE = r'#include "C:\Users\Baron\Desktop\EE_297_Repo\EE_297\hardware_imp\vitis_hls\lane_seg_hls\lane_seg_top.h"'

def write_header_plain_numbers(array: np.ndarray, var_name: str, ctype: str, out_path: str):
    array = np.asarray(array, dtype=np.float32)
    dims  = "".join(f"[{d}]" for d in array.shape)

    def fmt(a, indent=0):
        if a.ndim == 1:
            return "{" + ",".join(f"{x:.6f}" for x in a.tolist()) + "}"
        body = ",\n".join((" "*(indent+2)) + fmt(x, indent+2) for x in a)
        return "{\n" + body + "\n" + (" "*indent) + "}"

    with open(out_path, "w") as f:
        f.write("#pragma once\n\n")
        f.write(f"{HEADER_INCLUDE}\n\n")
        f.write("// Auto-generated with BN folded\n\n")
        f.write(f"{ctype} {var_name}{dims} = ")
        f.write(fmt(array))
        f.write(";\n")

    print("Wrote:", os.path.abspath(out_path))


os.makedirs(OUT_DIR, exist_ok=True)

# Expansion (1x1, 16→96)
write_header_plain_numbers(W_exp_hls, "enc2_ir1_exp_w", CTYPE, os.path.join(OUT_DIR, "enc2_ir1_exp_w.h"))
write_header_plain_numbers(b_exp_fold, "enc2_ir1_exp_b", CTYPE, os.path.join(OUT_DIR, "enc2_ir1_exp_b.h"))

# Depthwise (3x3, stride=2, groups=96)
write_header_plain_numbers(W_dw_hls, "enc2_ir1_dw_w", CTYPE, os.path.join(OUT_DIR, "enc2_ir1_dw_w.h"))
write_header_plain_numbers(b_dw_fold, "enc2_ir1_dw_b", CTYPE, os.path.join(OUT_DIR, "enc2_ir1_dw_b.h"))

# Projection (1x1, 96→24)
write_header_plain_numbers(W_pw_hls, "enc2_ir1_pw_w", CTYPE, os.path.join(OUT_DIR, "enc2_ir1_pw_w.h"))
write_header_plain_numbers(b_pw_fold, "enc2_ir1_pw_b", CTYPE, os.path.join(OUT_DIR, "enc2_ir1_pw_b.h"))


Wrote: C:\Users\Baron\Desktop\EE_297_Repo\EE_297\ML_PATH_EE297\EE297_env\01_main\03_lanes_code\weights\enc2_ir1_exp_w.h
Wrote: C:\Users\Baron\Desktop\EE_297_Repo\EE_297\ML_PATH_EE297\EE297_env\01_main\03_lanes_code\weights\enc2_ir1_exp_b.h
Wrote: C:\Users\Baron\Desktop\EE_297_Repo\EE_297\ML_PATH_EE297\EE297_env\01_main\03_lanes_code\weights\enc2_ir1_dw_w.h
Wrote: C:\Users\Baron\Desktop\EE_297_Repo\EE_297\ML_PATH_EE297\EE297_env\01_main\03_lanes_code\weights\enc2_ir1_dw_b.h
Wrote: C:\Users\Baron\Desktop\EE_297_Repo\EE_297\ML_PATH_EE297\EE297_env\01_main\03_lanes_code\weights\enc2_ir1_pw_w.h
Wrote: C:\Users\Baron\Desktop\EE_297_Repo\EE_297\ML_PATH_EE297\EE297_env\01_main\03_lanes_code\weights\enc2_ir1_pw_b.h


### (invertedresidual1): SW output of Conv Stage 2

In [21]:
# -------------------------------
# CONFIGURATION
# -------------------------------
STAGE1_OUT_PATH = "software_output/sw_encoder1_ir0_output.npy"
WEIGHT_SAVE_DIR = "software_output"
os.makedirs(WEIGHT_SAVE_DIR, exist_ok=True)

# -------------------------------
# LOAD PREVIOUS STAGE OUTPUT
# -------------------------------
x = np.load(STAGE1_OUT_PATH).astype(np.float32)   # shape (16,112,112)
x = np.expand_dims(x, axis=0)                     # (1,16,112,112)
x_torch = torch.tensor(x, dtype=torch.float32)

# -------------------------------
# Load model + weights
# -------------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = LaneSegOnlyMobileNetV2(pretrained=False)
model.load_state_dict(torch.load("lane_seg_weights.pth", map_location=device))
model.to(device)
model.eval()

# Extract encoder[2] (InvertedResidual)
ir1 = model.encoder[2]
convseq = ir1.conv

# ---- Expansion Conv + BN (1x1, 16→96) ----
exp_conv: torch.nn.Conv2d = convseq[0][0]
exp_bn:   torch.nn.BatchNorm2d = convseq[0][1]

W_exp = exp_conv.weight.detach().cpu().numpy().astype(np.float32)  # (96,16,1,1)
gamma_exp = exp_bn.weight.detach().cpu().numpy().astype(np.float32)
beta_exp  = exp_bn.bias.detach().cpu().numpy().astype(np.float32)
mean_exp  = exp_bn.running_mean.detach().cpu().numpy().astype(np.float32)
var_exp   = exp_bn.running_var.detach().cpu().numpy().astype(np.float32)
eps_exp   = float(exp_bn.eps)

s_exp = gamma_exp / np.sqrt(var_exp + eps_exp)
W_exp_fold = (W_exp * s_exp[:, None, None, None]).astype(np.float16)
b_exp_fold = (beta_exp - mean_exp * s_exp).astype(np.float16)

# ---- Depthwise Conv + BN (3x3, stride=2, groups=96) ----
dw_conv: torch.nn.Conv2d = convseq[1][0]
dw_bn:   torch.nn.BatchNorm2d = convseq[1][1]

W_dw = dw_conv.weight.detach().cpu().numpy().astype(np.float32)  # (96,1,3,3)
gamma_dw = dw_bn.weight.detach().cpu().numpy().astype(np.float32)
beta_dw  = dw_bn.bias.detach().cpu().numpy().astype(np.float32)
mean_dw  = dw_bn.running_mean.detach().cpu().numpy().astype(np.float32)
var_dw   = dw_bn.running_var.detach().cpu().numpy().astype(np.float32)
eps_dw   = float(dw_bn.eps)

s_dw = gamma_dw / np.sqrt(var_dw + eps_dw)
W_dw_fold = (W_dw * s_dw[:, None, None, None]).astype(np.float16)
b_dw_fold = (beta_dw - mean_dw * s_dw).astype(np.float16)

# ---- Projection Conv + BN (1x1, 96→24) ----
pw_conv: torch.nn.Conv2d = convseq[2]
pw_bn:   torch.nn.BatchNorm2d = convseq[3]

W_pw = pw_conv.weight.detach().cpu().numpy().astype(np.float32)  # (24,96,1,1)
gamma_pw = pw_bn.weight.detach().cpu().numpy().astype(np.float32)
beta_pw  = pw_bn.bias.detach().cpu().numpy().astype(np.float32)
mean_pw  = pw_bn.running_mean.detach().cpu().numpy().astype(np.float32)
var_pw   = pw_bn.running_var.detach().cpu().numpy().astype(np.float32)
eps_pw   = float(pw_bn.eps)

s_pw = gamma_pw / np.sqrt(var_pw + eps_pw)
W_pw_fold = (W_pw * s_pw[:, None, None, None]).astype(np.float16)
b_pw_fold = (beta_pw - mean_pw * s_pw).astype(np.float16)

# Convert to torch tensors
W_exp_torch = torch.tensor(W_exp_fold, dtype=torch.float16, device=device)
b_exp_torch = torch.tensor(b_exp_fold, dtype=torch.float16, device=device)
W_dw_torch  = torch.tensor(W_dw_fold,  dtype=torch.float16, device=device)
b_dw_torch  = torch.tensor(b_dw_fold,  dtype=torch.float16, device=device)
W_pw_torch  = torch.tensor(W_pw_fold,  dtype=torch.float16, device=device)
b_pw_torch  = torch.tensor(b_pw_fold,  dtype=torch.float16, device=device)

x_torch = x_torch.to(device).to(torch.float16)

# -------------------------------
# Run stage 2 (expand → depthwise → projection)
# -------------------------------
with torch.no_grad():
    # Expansion
    y = F.conv2d(x_torch, W_exp_torch, bias=b_exp_torch, stride=1, padding=0)
    y = torch.clamp(y, 0, 6)  # ReLU6

    # Depthwise
    y = F.conv2d(y, W_dw_torch, bias=b_dw_torch, stride=2, padding=1, groups=96)
    y = torch.clamp(y, 0, 6)  # ReLU6

    # Projection
    y = F.conv2d(y, W_pw_torch, bias=b_pw_torch, stride=1, padding=0)
    # no activation here

# Save results
y_np = y.squeeze(0).cpu().numpy()  # (24,56,56)
np.save(os.path.join(WEIGHT_SAVE_DIR, "sw_encoder2_ir1_output.npy"), y_np)

with open(os.path.join(WEIGHT_SAVE_DIR, "sw_encoder2_ir1_output.txt"), "w") as f:
    for c in range(24):
        for i in range(56):
            for j in range(56):
                f.write(f"{y_np[c][i][j]:.6f} ")
            f.write("\n")
        f.write("\n")

print("✅ Software output for enc2_ir1 saved.")


C:\Users\Baron\AppData\Local\Temp\ipykernel_14072\4104754890.py:20: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("lane_seg_weights.pth", ma

✅ Software output for enc2_ir1 saved.


In [24]:
# === enc2_ir1 single-pixel debug (oy=0, ox=0), HLS-faithful ===
import numpy as np

# Pull raw arrays
x_np  = x_torch.squeeze(0).detach().cpu().numpy()            # (16,112,112)
Wexp  = W_exp_torch.detach().cpu().numpy()                   # (96,16,1,1)
bexp  = b_exp_torch.detach().cpu().numpy()                   # (96,)
Wdw   = W_dw_torch.detach().cpu().numpy()                    # (96,1,3,3)
bdw   = b_dw_torch.detach().cpu().numpy()                    # (96,)
Wpw   = W_pw_torch.detach().cpu().numpy()                    # (24,96,1,1)
bpw   = b_pw_torch.detach().cpu().numpy()                    # (24,)

IN_C   = x_np.shape[0]   # 16
IN_H   = x_np.shape[1]   # 112
IN_W   = x_np.shape[2]   # 112
EXP_C  = Wexp.shape[0]   # 96
OUT_C  = Wpw.shape[0]    # 24
K = 3

oy = ox = 0
iy0 = oy * 2
ix0 = ox * 2

# ---- Expansion: compute exp_out at *all positions* (not just (0,0)) ----
exp_out = np.zeros((EXP_C, IN_H, IN_W), dtype=np.float32)
for oc in range(EXP_C):
    for iy in range(IN_H):
        for ix in range(IN_W):
            s = bexp[oc]
            for ic in range(IN_C):
                s += x_np[ic, iy, ix] * Wexp[oc, ic, 0, 0]
            # ReLU6
            s = np.clip(s, 0, 6)
            exp_out[oc, iy, ix] = s

# Debug one pixel/channel
print(f"EXP DEBUG: output[0][0][0] = {float(exp_out[0,0,0]):.6f}")

# ---- Depthwise: use proper 3x3 neighborhood of expanded feature map ----
dw_out00 = np.zeros((EXP_C,), dtype=np.float32)
for c in range(EXP_C):
    s = bdw[c]
    if c == 0:
        print(f"DW DEBUG: bias[{c}] = {float(s):.6f}")
    for ky in range(K):
        for kx in range(K):
            iy = iy0 + ky - 1
            ix = ix0 + kx - 1
            if 0 <= iy < IN_H and 0 <= ix < IN_W:
                a = exp_out[c, iy, ix]
                b = Wdw[c, 0, ky, kx]
                p = a * b
                s += p
                if c == 0:
                    print(f"DW DEBUG: a={float(a):.6f} * b={float(b):.6f} => {float(p):.6f} | sum={float(s):.6f}")
    # ReLU6
    s = np.clip(s, 0, 6)
    dw_out00[c] = s
    if c == 0:
        print(f"DW DEBUG: output[{oy}][{ox}][{c}] after ReLU6 = {float(s):.6f}")

# ---- Projection: oc=0 at same pixel ----
oc_pw = 0
sum_pw = bpw[oc_pw]
print(f"PW DEBUG: bias[{oc_pw}] = {float(sum_pw):.6f}")
for ic in range(EXP_C):
    a = dw_out00[ic]
    b = Wpw[oc_pw, ic, 0, 0]
    p = a * b
    sum_pw += p
    if ic < 3:
        print(f"PW DEBUG: ic={ic}, a={float(a):.6f}, b={float(b):.6f}, p={float(p):.6f}, sum={float(sum_pw):.6f}")

print(f"PW DEBUG: output[{oy}][{ox}][{oc_pw}] final = {float(sum_pw):.6f}")


EXP DEBUG: output[0][0][0] = 1.440430
DW DEBUG: bias[0] = 0.371826
DW DEBUG: a=1.440430 * b=-0.578125 => -0.832748 | sum=-0.460922
DW DEBUG: a=1.109375 * b=-0.298340 => -0.330971 | sum=-0.791893
DW DEBUG: a=0.956055 * b=-0.311035 => -0.297367 | sum=-1.089260
DW DEBUG: a=0.686523 * b=-0.141846 => -0.097380 | sum=-1.186640
DW DEBUG: output[0][0][0] after ReLU6 = 0.000000
PW DEBUG: bias[0] = -1.168945
PW DEBUG: ic=0, a=0.000000, b=0.259033, p=0.000000, sum=-1.168945
PW DEBUG: ic=1, a=0.386230, b=0.259033, p=0.100047, sum=-1.068899
PW DEBUG: ic=2, a=0.000000, b=-0.005741, p=-0.000000, sum=-1.068899
PW DEBUG: output[0][0][0] final = 0.302640


### (invertedresidual2): HW output of Conv Stage 3
     1. Save Weights File

In [25]:
# 1) Get encoder stage (InvertedResidual at index 3 for enc3_ir2)
ir2 = model.encoder[3]        # InvertedResidual (your "enc3_ir2")
convseq = ir2.conv            # Sequential inside it

# ---- Expansion Conv + BN (1x1, 24→144) ----
exp_conv: nn.Conv2d = convseq[0][0]
exp_bn:   nn.BatchNorm2d = convseq[0][1]

W_exp = exp_conv.weight.detach().cpu().numpy().astype("float32")  # (144,24,1,1)
gamma_exp = exp_bn.weight.detach().cpu().numpy().astype("float32")
beta_exp  = exp_bn.bias.detach().cpu().numpy().astype("float32")
mean_exp  = exp_bn.running_mean.detach().cpu().numpy().astype("float32")
var_exp   = exp_bn.running_var.detach().cpu().numpy().astype("float32")
eps_exp   = float(exp_bn.eps)

s_exp = (gamma_exp / np.sqrt(var_exp + eps_exp)).astype("float32")
W_exp_fold = (W_exp * s_exp[:, None, None, None]).astype("float32")
b_exp_fold = (beta_exp - mean_exp * s_exp).astype("float32")

# Layout for HLS: (K,K,IN_C,OUT_C) → (1,1,24,144)
W_exp_hls = np.transpose(W_exp_fold, (2, 3, 1, 0))

# ---- Depthwise Conv + BN (3x3, stride=1, groups=144) ----
dw_conv: nn.Conv2d = convseq[1][0]
dw_bn:   nn.BatchNorm2d = convseq[1][1]

W_dw = dw_conv.weight.detach().cpu().numpy().astype("float32")  # (144,1,3,3)
gamma_dw = dw_bn.weight.detach().cpu().numpy().astype("float32")
beta_dw  = dw_bn.bias.detach().cpu().numpy().astype("float32")
mean_dw  = dw_bn.running_mean.detach().cpu().numpy().astype("float32")
var_dw   = dw_bn.running_var.detach().cpu().numpy().astype("float32")
eps_dw   = float(dw_bn.eps)

s_dw = (gamma_dw / np.sqrt(var_dw + eps_dw)).astype("float32")
W_dw_fold = (W_dw * s_dw[:, None, None, None]).astype("float32")
b_dw_fold = (beta_dw - mean_dw * s_dw).astype("float32")

# Layout for HLS: (K,K,IN_C,OUT_C) — depthwise has groups=144, so IN=1 per channel
W_dw_hls = np.transpose(W_dw_fold, (2, 3, 1, 0))  # (3,3,1,144)

# ---- Projection Conv + BN (1x1, 144→24) ----
pw_conv: nn.Conv2d = convseq[2]
pw_bn:   nn.BatchNorm2d = convseq[3]

W_pw = pw_conv.weight.detach().cpu().numpy().astype("float32")  # (24,144,1,1)
gamma_pw = pw_bn.weight.detach().cpu().numpy().astype("float32")
beta_pw  = pw_bn.bias.detach().cpu().numpy().astype("float32")
mean_pw  = pw_bn.running_mean.detach().cpu().numpy().astype("float32")
var_pw   = pw_bn.running_var.detach().cpu().numpy().astype("float32")
eps_pw   = float(pw_bn.eps)

s_pw = (gamma_pw / np.sqrt(var_pw + eps_pw)).astype("float32")
W_pw_fold = (W_pw * s_pw[:, None, None, None]).astype("float32")
b_pw_fold = (beta_pw - mean_pw * s_pw).astype("float32")

# Layout for HLS: (K,K,IN_C,OUT_C) → (1,1,144,24)
W_pw_hls = np.transpose(W_pw_fold, (2, 3, 1, 0))


In [26]:
import os
import numpy as np

OUT_DIR = "weights"
CTYPE   = "data_t"  # ap_fixed typedef in HLS

HEADER_INCLUDE = r'#include "C:\Users\Baron\Desktop\EE_297_Repo\EE_297\hardware_imp\vitis_hls\lane_seg_hls\lane_seg_top.h"'

def write_header_plain_numbers(array: np.ndarray, var_name: str, ctype: str, out_path: str):
    array = np.asarray(array, dtype=np.float32)
    dims  = "".join(f"[{d}]" for d in array.shape)

    def fmt(a, indent=0):
        if a.ndim == 1:
            return "{" + ",".join(f"{x:.6f}" for x in a.tolist()) + "}"
        body = ",\n".join((" "*(indent+2)) + fmt(x, indent+2) for x in a)
        return "{\n" + body + "\n" + (" "*indent) + "}"

    with open(out_path, "w") as f:
        f.write("#pragma once\n\n")
        f.write(f"{HEADER_INCLUDE}\n\n")
        f.write("// Auto-generated with BN folded\n\n")
        f.write(f"{ctype} {var_name}{dims} = ")
        f.write(fmt(array))
        f.write(";\n")

    print("Wrote:", os.path.abspath(out_path))


os.makedirs(OUT_DIR, exist_ok=True)

# Expansion (1x1, 24→144)
write_header_plain_numbers(W_exp_hls, "enc3_ir2_exp_w", CTYPE, os.path.join(OUT_DIR, "enc3_ir2_exp_w.h"))
write_header_plain_numbers(b_exp_fold, "enc3_ir2_exp_b", CTYPE, os.path.join(OUT_DIR, "enc3_ir2_exp_b.h"))

# Depthwise (3x3, stride=1, groups=144)
write_header_plain_numbers(W_dw_hls, "enc3_ir2_dw_w", CTYPE, os.path.join(OUT_DIR, "enc3_ir2_dw_w.h"))
write_header_plain_numbers(b_dw_fold, "enc3_ir2_dw_b", CTYPE, os.path.join(OUT_DIR, "enc3_ir2_dw_b.h"))

# Projection (1x1, 144→24)
write_header_plain_numbers(W_pw_hls, "enc3_ir2_pw_w", CTYPE, os.path.join(OUT_DIR, "enc3_ir2_pw_w.h"))
write_header_plain_numbers(b_pw_fold, "enc3_ir2_pw_b", CTYPE, os.path.join(OUT_DIR, "enc3_ir2_pw_b.h"))


Wrote: C:\Users\Baron\Desktop\EE_297_Repo\EE_297\ML_PATH_EE297\EE297_env\01_main\03_lanes_code\weights\enc3_ir2_exp_w.h
Wrote: C:\Users\Baron\Desktop\EE_297_Repo\EE_297\ML_PATH_EE297\EE297_env\01_main\03_lanes_code\weights\enc3_ir2_exp_b.h
Wrote: C:\Users\Baron\Desktop\EE_297_Repo\EE_297\ML_PATH_EE297\EE297_env\01_main\03_lanes_code\weights\enc3_ir2_dw_w.h
Wrote: C:\Users\Baron\Desktop\EE_297_Repo\EE_297\ML_PATH_EE297\EE297_env\01_main\03_lanes_code\weights\enc3_ir2_dw_b.h
Wrote: C:\Users\Baron\Desktop\EE_297_Repo\EE_297\ML_PATH_EE297\EE297_env\01_main\03_lanes_code\weights\enc3_ir2_pw_w.h
Wrote: C:\Users\Baron\Desktop\EE_297_Repo\EE_297\ML_PATH_EE297\EE297_env\01_main\03_lanes_code\weights\enc3_ir2_pw_b.h


### (invertedresidual2): SW output of Conv Stage 3

In [27]:
# -------------------------------
# CONFIGURATION
# -------------------------------
STAGE2_OUT_PATH = "software_output/sw_encoder2_ir1_output.npy"
WEIGHT_SAVE_DIR = "software_output"
os.makedirs(WEIGHT_SAVE_DIR, exist_ok=True)

# -------------------------------
# LOAD PREVIOUS STAGE OUTPUT
# -------------------------------
x = np.load(STAGE2_OUT_PATH).astype(np.float32)   # shape (24,56,56)
x = np.expand_dims(x, axis=0)                     # (1,24,56,56)
x_torch = torch.tensor(x, dtype=torch.float32)

# -------------------------------
# Load model + weights
# -------------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = LaneSegOnlyMobileNetV2(pretrained=False)
model.load_state_dict(torch.load("lane_seg_weights.pth", map_location=device))
model.to(device)
model.eval()

# Extract encoder[3] (InvertedResidual)
ir2 = model.encoder[3]
convseq = ir2.conv

# ---- Expansion Conv + BN (1x1, 24→144) ----
exp_conv: torch.nn.Conv2d = convseq[0][0]
exp_bn:   torch.nn.BatchNorm2d = convseq[0][1]

W_exp = exp_conv.weight.detach().cpu().numpy().astype(np.float32)  # (144,24,1,1)
gamma_exp = exp_bn.weight.detach().cpu().numpy().astype(np.float32)
beta_exp  = exp_bn.bias.detach().cpu().numpy().astype(np.float32)
mean_exp  = exp_bn.running_mean.detach().cpu().numpy().astype(np.float32)
var_exp   = exp_bn.running_var.detach().cpu().numpy().astype(np.float32)
eps_exp   = float(exp_bn.eps)

s_exp = gamma_exp / np.sqrt(var_exp + eps_exp)
W_exp_fold = (W_exp * s_exp[:, None, None, None]).astype(np.float16)
b_exp_fold = (beta_exp - mean_exp * s_exp).astype(np.float16)

# ---- Depthwise Conv + BN (3x3, stride=1, groups=144) ----
dw_conv: torch.nn.Conv2d = convseq[1][0]
dw_bn:   torch.nn.BatchNorm2d = convseq[1][1]

W_dw = dw_conv.weight.detach().cpu().numpy().astype(np.float32)  # (144,1,3,3)
gamma_dw = dw_bn.weight.detach().cpu().numpy().astype(np.float32)
beta_dw  = dw_bn.bias.detach().cpu().numpy().astype(np.float32)
mean_dw  = dw_bn.running_mean.detach().cpu().numpy().astype(np.float32)
var_dw   = dw_bn.running_var.detach().cpu().numpy().astype(np.float32)
eps_dw   = float(dw_bn.eps)

s_dw = gamma_dw / np.sqrt(var_dw + eps_dw)
W_dw_fold = (W_dw * s_dw[:, None, None, None]).astype(np.float16)
b_dw_fold = (beta_dw - mean_dw * s_dw).astype(np.float16)

# ---- Projection Conv + BN (1x1, 144→24) ----
pw_conv: torch.nn.Conv2d = convseq[2]
pw_bn:   torch.nn.BatchNorm2d = convseq[3]

W_pw = pw_conv.weight.detach().cpu().numpy().astype(np.float32)  # (24,144,1,1)
gamma_pw = pw_bn.weight.detach().cpu().numpy().astype(np.float32)
beta_pw  = pw_bn.bias.detach().cpu().numpy().astype(np.float32)
mean_pw  = pw_bn.running_mean.detach().cpu().numpy().astype(np.float32)
var_pw   = pw_bn.running_var.detach().cpu().numpy().astype(np.float32)
eps_pw   = float(pw_bn.eps)

s_pw = gamma_pw / np.sqrt(var_pw + eps_pw)
W_pw_fold = (W_pw * s_pw[:, None, None, None]).astype(np.float16)
b_pw_fold = (beta_pw - mean_pw * s_pw).astype(np.float16)

# Convert to torch tensors
W_exp_torch = torch.tensor(W_exp_fold, dtype=torch.float16, device=device)
b_exp_torch = torch.tensor(b_exp_fold, dtype=torch.float16, device=device)
W_dw_torch  = torch.tensor(W_dw_fold,  dtype=torch.float16, device=device)
b_dw_torch  = torch.tensor(b_dw_fold,  dtype=torch.float16, device=device)
W_pw_torch  = torch.tensor(W_pw_fold,  dtype=torch.float16, device=device)
b_pw_torch  = torch.tensor(b_pw_fold,  dtype=torch.float16, device=device)

x_torch = x_torch.to(device).to(torch.float16)

# -------------------------------
# Run stage 3 (expand → depthwise → projection)
# -------------------------------
with torch.no_grad():
    # Expansion
    y = F.conv2d(x_torch, W_exp_torch, bias=b_exp_torch, stride=1, padding=0)
    y = torch.clamp(y, 0, 6)  # ReLU6

    # Depthwise
    y = F.conv2d(y, W_dw_torch, bias=b_dw_torch, stride=1, padding=1, groups=144)
    y = torch.clamp(y, 0, 6)  # ReLU6

    # Projection
    y = F.conv2d(y, W_pw_torch, bias=b_pw_torch, stride=1, padding=0)
    # no activation here

# Save results
y_np = y.squeeze(0).cpu().numpy()  # (24,56,56)
np.save(os.path.join(WEIGHT_SAVE_DIR, "sw_encoder3_ir2_output.npy"), y_np)

with open(os.path.join(WEIGHT_SAVE_DIR, "sw_encoder3_ir2_output.txt"), "w") as f:
    for c in range(24):
        for i in range(56):
            for j in range(56):
                f.write(f"{y_np[c][i][j]:.6f} ")
            f.write("\n")
        f.write("\n")

print("✅ Software output for enc3_ir2 saved.")


C:\Users\Baron\AppData\Local\Temp\ipykernel_14072\454475225.py:20: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("lane_seg_weights.pth", map

✅ Software output for enc3_ir2 saved.


### (invertedresidual3): HW output of Conv Stage 4
     1. Save Weights File

### (invertedresidual3): SW output of Conv Stage 4

### (invertedresidual4): HW output of Conv Stage 5
     1. Save Weights File

### (invertedresidual4): SW output of Conv Stage 5

### UNUSED CODE